# BentoML Example:  Deploy to AWS SageMaker

[BentoML](http://bentoml.ai) is an open source framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

This notebook demonstrates how to use BentoML to deploy a machine learning model as a REST API endpoint to AWS SageMaker. For this demo, we are using the [Sentiment Analysis with Scikit-learn](https://github.com/bentoml/BentoML/blob/master/examples/sklearn-sentiment-clf/sklearn-sentiment-clf.ipynb) example, using dataset from [Sentiment140](http://help.sentiment140.com/for-students/).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=nb&ea=open&el=official-example&dt=deploy-with-sagemaker)

In [ ]:
!pip install -I bentoml
!pip install sklearn pandas numpy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

import bentoml

# Prepare Dataset

In [ ]:
%%bash

if [ ! -f ./trainingandtestdata.zip ]; then
    wget -q http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    unzip -n trainingandtestdata.zip
fi

In [ ]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns

# Model Training

In [ ]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df = 100,
                                                        ngram_range = (1,1),
                                                        stop_words = 'english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(dftrain.text, dftrain.polarity)

In [ ]:
Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]
print(classification_report(ytest,sentiment_lr.predict(Xtest)))

In [ ]:
sentiment_lr.predict([Xtest[0]])

# Define ML Service with BentoML

In [ ]:
%%writefile sentiment_lr_model.py
import pandas as pd
import bentoml
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([PickleArtifact('sentiment_lr')])
@bentoml.env(pip_dependencies=["scikit-learn", "pandas"])
class SentimentLRModel(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler, typ='series')
    def predict(self, series):
        """
        predict expects pandas.Series as input
        """        
        return self.artifacts.sentiment_lr.predict(series)

# Save BentoML service archive

In [ ]:
from sentiment_lr_model import SentimentLRModel

# Initialize bentoML model with artifacts

bento_model = SentimentLRModel.pack(
    sentiment_lr=sentiment_lr
)

# Save bentoML model to directory
saved_path = bento_model.save()

# print the directory containing exported model archive (prefixed with model name and version)
print(saved_path)

# Load BentoML Service from archive

In [ ]:
import bentoml

# Load exported bentoML model archive from path
bento_model = bentoml.load(saved_path)

# Call predict on the restored sklearn model
bento_model.predict(pd.Series(["hello", "hi"]))

# Deploy to AWS SageMaker

```bash
bentoml deployment create DEPLOYMENT_NAME --bento=BENTO_TAG --platform=PLATFORM --region=REGION --api-name=API_NAME
```
### Arguments:
* deployment name

### Options:
* bento_tag: REQUIRED. The Bento model service, in the format of name:version.
* platform: REQUIRED.  The platform that you want to deploy bentoml bundle to.  For serverless, we support aws-lambda, aws-lambda-py2 and gcp-function.
* api_name: REQUIRED. API function will be used for Sagemaker
* region: OPTIONAL The cloud provider's region you want to deploy in.


In [ ]:
bento_tag = '{name}:{version}'.format(name=bento_model.name, version=bento_model.version)

In [ ]:
!bentoml deployment create sentiment-sagemaker --bento {bento_tag} --platform aws-sagemaker --api-name predict

### Make request against the endpoint

We are using AWS CLI to make prediction request with SageMaker endpoint. The result will stored in `output.json` file.

```bash
aws sagemaker-runtime invoke-endpoint \
--endpoint-name default-SentimentLRModel \
--body '["new food", "bad movie", "chicken nuggets", "good family time"]' \
--content-type "application/json"\
output.json
```

In [ ]:
!aws sagemaker-runtime invoke-endpoint \
--endpoint-name default-SentimentLRModel \
--body '["new food", "bad movie", "chicken nuggets", "good family time"]' \
--content-type "application/json" \
output.json
!cat output.json

# Check deployment status

```
bentoml deployment describe DEPLOYMENT_NAME --namespace NAMESPACE
```

### Arguments:
* deployment name

### Options:
* namespace: OPTIONAL.  Namespace for the deployment

In [ ]:
!bentoml deployment describe sentiment-sagemaker

# Delete serverless Deployment


```bash
bentoml deployment delete DEPLOYMENT_NAME --namespace NAMESPACE
```

### Arguments:
* deployment name

### Options:
* namespace: OPTIONAL.  Namespace for the deployment

In [ ]:
!bentoml deployment delete sentiment-sagemaker